# Notebook for getting connections dataframe


In [5]:
import pyspark.sql.functions as F
import math
import pandas as pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Start Spark

In [1]:
%%configure
{"conf": {
    "spark.app.name": "datavirus_final"
}}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7704,application_1589299642358_2200,pyspark,idle,Link,Link,
7735,application_1589299642358_2231,pyspark,busy,Link,Link,
7737,application_1589299642358_2233,pyspark,idle,Link,Link,
7739,application_1589299642358_2235,pyspark,dead,Link,Link,
7743,application_1589299642358_2239,pyspark,idle,Link,Link,
7745,application_1589299642358_2241,pyspark,idle,Link,Link,
7750,application_1589299642358_2246,pyspark,busy,Link,Link,
7753,application_1589299642358_2249,pyspark,idle,Link,Link,
7756,application_1589299642358_2252,pyspark,idle,Link,Link,
7759,application_1589299642358_2255,pyspark,busy,Link,Link,


In [2]:
# Initialization
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7783,application_1589299642358_2280,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7704,application_1589299642358_2200,pyspark,idle,Link,Link,
7735,application_1589299642358_2231,pyspark,idle,Link,Link,
7737,application_1589299642358_2233,pyspark,idle,Link,Link,
7739,application_1589299642358_2235,pyspark,dead,Link,Link,
7743,application_1589299642358_2239,pyspark,idle,Link,Link,
7745,application_1589299642358_2241,pyspark,idle,Link,Link,
7750,application_1589299642358_2246,pyspark,busy,Link,Link,
7753,application_1589299642358_2249,pyspark,idle,Link,Link,
7756,application_1589299642358_2252,pyspark,idle,Link,Link,
7759,application_1589299642358_2255,pyspark,busy,Link,Link,


In [3]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Compute probabilities and average delays using all possible data

In [77]:
# Load SBB data only for stations within 15 km of Zurich
#ids_stations = spark.read.csv('../data/zurich_stations_ids.csv')
#sbb_zurich = sbb.join(ids_stations,sbb['BPUIC']==ids_stations['_c0']).drop("_c0")

sbb = spark.read.orc('/data/sbb/orc/istdaten')
sample = sbb.where(F.col('FAHRT_BEZEICHNER')=='85:3849:140841-04004-1')\
            #.where((F.col('BETRIEBSTAG')=='07.07.2018') | (F.col('BETRIEBSTAG')=='03.03.2018') | (F.col('BETRIEBSTAG')=='28.04.2018') | (F.col('BETRIEBSTAG')=='17.11.2018'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
simple_df = (
    sample
    .select('ankunftszeit','an_prognose','abfahrtszeit','BETRIEBSTAG','FAHRT_BEZEICHNER','LINIEN_ID','PRODUKT_ID','BPUIC','HALTESTELLEN_NAME')
    .toDF('Arrival_Time','Arrival_Real','Departure_Time','Day','Trip_ID','Line_ID', 'Type','Station_ID', 'Station_Name')
    .dropDuplicates()
    .withColumn('arrival', F.unix_timestamp(F.col('Arrival_Time'), "dd.MM.yyyy HH:mm").cast('long'))
    .withColumn('real_arrival', F.unix_timestamp(F.col('Arrival_Real'), "dd.MM.yyyy HH:mm:ss").cast("long"))
    .withColumn('Arrival_Delay',F.col('real_arrival')-F.col('arrival')).drop('arrival','real_arrival')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
# Get a simpler dataframe with delays for each trip and without duplicated rows
# Solve null values: Remove columns with more than two null-values
# and Copy "Arrival_Time" value if "Departure_Time" is null and viceversa

delays_df = (
    simple_df 
    .withColumn('Arrival_null', F.when(F.col('Arrival_Time') == '', None).otherwise(F.col('Arrival_Time')))
    .withColumn('Departure_null', F.when(F.col('Departure_Time') == '', None).otherwise(F.col('Departure_Time')))
    .drop('Arrival_Time','Departure_Time')
    .dropna(thresh=1,subset=('Arrival_null','Departure_null'))
    .withColumn("Departure",F.coalesce(F.col('Arrival_null'),F.col('Departure_null')))\
    .withColumn("Arrival", F.coalesce(F.col('Departure_null'),F.col('Arrival_null')))\
    .drop('Arrival_null','Departure_null')
    .orderBy('Trip_ID','Day','Arrival','Departure')

)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
delays_df.show(50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+--------------------+-----------+----+----------+--------------------+-------------+----------------+----------------+
|       Arrival_Real|       Day|             Trip_ID|    Line_ID|Type|Station_ID|        Station_Name|Arrival_Delay|       Departure|         Arrival|
+-------------------+----------+--------------------+-----------+----+----------+--------------------+-------------+----------------+----------------+
|                   |01.09.2018|85:3849:140841-04...|85:3849:004|Tram|   8576182|Zürich, Bahnhof T...|         null|01.09.2018 11:47|01.09.2018 11:47|
|01.09.2018 11:49:05|01.09.2018|85:3849:140841-04...|85:3849:004|Tram|   8576200|Zürich, Wildbachs...|           65|01.09.2018 11:48|01.09.2018 11:48|
|01.09.2018 11:49:59|01.09.2018|85:3849:140841-04...|85:3849:004|Tram|   8576199|Zürich, Fröhlichs...|           59|01.09.2018 11:49|01.09.2018 11:49|
|01.09.2018 11:51:11|01.09.2018|85:3849:140841-04...|85:3849:004|Tram|   8576198|  Zürich, Hös

In [91]:
delays_df.where(delays_df.Arrival.substr(12,5)=='11:55').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+--------------------+-----------+----+----------+--------------------+-------------+----------------+----------------+
|       Arrival_Real|       Day|             Trip_ID|    Line_ID|Type|Station_ID|        Station_Name|Arrival_Delay|       Departure|         Arrival|
+-------------------+----------+--------------------+-----------+----+----------+--------------------+-------------+----------------+----------------+
|07.07.2018 11:55:12|07.07.2018|85:3849:140841-04...|85:3849:004|Tram|   8576199|Zürich, Fröhlichs...|           12|07.07.2018 11:55|07.07.2018 11:55|
|07.07.2018 11:55:12|07.07.2018|85:3849:140841-04...|85:3849:004|Tram|   8576195|   Zürich, Opernhaus|           12|07.07.2018 11:55|07.07.2018 11:55|
|07.07.2018 11:55:12|07.07.2018|85:3849:140841-04...|85:3849:004|Tram|   8576196|Zürich, Kreuzstrasse|           12|07.07.2018 11:55|07.07.2018 11:55|
|07.07.2018 11:55:12|07.07.2018|85:3849:140841-04...|85:3849:004|Tram|   8576198|  Zürich, Hös

In [17]:
# Remove rows corresponding to trips appearing only once on the dataframe
# These removable trips actually come from / go to stations outside Zurich
ids_counts = simple_df.groupBy('Trip_ID','Day').count()
ids_trips = ids_counts.where(ids_counts['count']>1).select('Trip_ID').distinct()
df = delays_df.join(ids_trips, "Trip_ID").orderBy('Trip_ID','Arrival','Departure')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
#Compute total number of connections (trip and station)
df = delays_df.fillna(0)

df_total_trips = df.groupBy('Trip_ID','Station_Name')\
                   .count().toDF('Trip_ID','Station_Name','Total')

#Compute number of delayed connections (trip and station)
df_delayed_trips = df.where(df['Arrival_Delay']>0).groupBy('Trip_ID','Station_Name')\
                    .count().toDF('Trip_ID','Station_Name','Delayed')

#Compute average delay for every connection (trip and station)
df_mean_delays = df.where(df['Arrival_Delay']>0).groupBy('Trip_ID','Station_Name')\
                .agg(F.mean('Arrival_Delay').alias("Mean_Delay"))

# Join dataframes
df_prob = df_total_trips.join(df_delayed_trips, on = ['Trip_ID','Station_Name'],how='left')\
            .withColumn("Probability", F.col("Delayed")/F.col("Total"))

df_prob_and_delays = df_prob.join(df_mean_delays, on = ['Trip_ID','Station_Name'],how='left')

df_final = df.join(df_prob_and_delays,on =['Trip_ID','Station_Name'],how='left').drop('Arrival_Delay')\
            .orderBy('Trip_ID','Day','Arrival').dropDuplicates(['Station_Name','Trip_ID','Mean_Delay'])\
            .orderBy('Trip_ID','Day','Arrival').fillna(0).cache()
df_final.show()
#better.stop_id.substr(0, 7)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-------------------+----------+-----------+----+----------+----------------+----------------+-----+-------+------------------+-----------------+
|             Trip_ID|        Station_Name|       Arrival_Real|       Day|    Line_ID|Type|Station_ID|       Departure|         Arrival|Total|Delayed|       Probability|       Mean_Delay|
+--------------------+--------------------+-------------------+----------+-----------+----+----------+----------------+----------------+-----+-------+------------------+-----------------+
|85:3849:140841-04...|Zürich, Bahnhof T...|                   |01.09.2018|85:3849:004|Tram|   8576182|01.09.2018 11:47|01.09.2018 11:47|   45|      0|               0.0|              0.0|
|85:3849:140841-04...|Zürich, Wildbachs...|01.09.2018 11:49:05|01.09.2018|85:3849:004|Tram|   8576200|01.09.2018 11:48|01.09.2018 11:48|   45|     44|0.9777777777777777|46.56818181818182|
|85:3849:140841-04...|Zürich, Fröhlichs...|01.09.2018 11:49:

In [82]:
df_final.where(df_final.Arrival.substr(12,5)=='11:55').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-------------------+----------+-----------+----+----------+----------------+----------------+-----+-------+-----------+----------+
|             Trip_ID|        Station_Name|       Arrival_Real|       Day|    Line_ID|Type|Station_ID|       Departure|         Arrival|Total|Delayed|Probability|Mean_Delay|
+--------------------+--------------------+-------------------+----------+-----------+----+----------+----------------+----------------+-----+-------+-----------+----------+
|85:3849:140841-04...|Zürich, Bahnhof S...|                   |07.07.2018|85:3849:004|Tram|   8503059|07.07.2018 11:55|07.07.2018 11:55|    1|      0|        0.0|       0.0|
|85:3849:140841-04...|   Zürich, Kunsthaus|23.06.2018 12:02:16|23.06.2018|85:3849:004|Tram|   8591239|23.06.2018 11:55|23.06.2018 11:55|    2|      1|        0.5|     436.0|
+--------------------+--------------------+-------------------+----------+-----------+----+----------+----------------+-----------

In [86]:
from pyspark.sql import Window
trip_window = Window.partitionBy('trip_id').orderBy(F.asc('Arrival'))
trip_rank = F.rank().over(trip_window).alias('stop')
df = df_final.select('*', trip_rank).alias('begin').orderBy('Trip_ID','Arrival')
to_station = df.drop('Departure').withColumn('stop', df.stop -1).alias('to_station')
from_station = df.select('stop','Trip_ID','Station_ID','Departure').join(to_station, on=['stop','Trip_ID']).orderBy('Trip_ID','stop').drop('Day','Line_ID','Station_Name')
#connections = from_station.toDF("Stop_sequence","Trip_ID","Start_Time","Start_Station",
                                #"Stop_Time","Stop_Station","Probability_Delay","Mean_Delay")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
from_station.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+----------+----------------+-------------------+----+----------+----------------+-----+-------+------------------+-----------------+
|stop|             Trip_ID|Station_ID|       Departure|       Arrival_Real|Type|Station_ID|         Arrival|Total|Delayed|       Probability|       Mean_Delay|
+----+--------------------+----------+----------------+-------------------+----+----------+----------------+-----+-------+------------------+-----------------+
|   1|85:3849:140841-04...|   8576182|01.09.2018 11:47|01.09.2018 11:49:05|Tram|   8576200|01.09.2018 11:48|   45|     44|0.9777777777777777|46.56818181818182|
|   2|85:3849:140841-04...|   8576200|01.09.2018 11:48|01.09.2018 11:49:59|Tram|   8576199|01.09.2018 11:49|   45|     44|0.9777777777777777|41.84090909090909|
|   3|85:3849:140841-04...|   8576199|01.09.2018 11:49|01.09.2018 11:52:11|Tram|   8576197|01.09.2018 11:51|   45|     43|0.9555555555555556|52.41860465116279|
|   4|85:3849:140841-04...|   8576197|01

In [89]:
from_station.where(from_station.Arrival.substr(12,5)=='11:55').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+----------+----------------+-------------------+----+----------+----------------+-----+-------+-----------+----------+
|stop|             Trip_ID|Station_ID|       Departure|       Arrival_Real|Type|Station_ID|         Arrival|Total|Delayed|Probability|Mean_Delay|
+----+--------------------+----------+----------------+-------------------+----+----------+----------------+-----+-------+-----------+----------+
|  26|85:3849:140841-04...|   8591309|03.11.2018 11:58|                   |Tram|   8503059|07.07.2018 11:55|    1|      0|        0.0|       0.0|
|  27|85:3849:140841-04...|   8503059|07.07.2018 11:55|23.06.2018 12:02:16|Tram|   8591239|23.06.2018 11:55|    2|      1|        0.5|     436.0|
+----+--------------------+----------+----------------+-------------------+----+----------+----------------+-----+-------+-----------+----------+

### Create final dataframe showing delay probabilities for each connection (in a normal workday)

In [92]:
#Filter data to trips happening only a single working day between 5h and 21h 
#df_day =df_final.where(F.col('Day')=='15.05.2019') 
df_min_hour = df_final.where(F.hour(F.unix_timestamp(F.col('Arrival'), "dd.MM.yyyy HH:mm").cast('timestamp'))>=5) 
df_max_hour = df_min_hour.where(F.hour(F.unix_timestamp(F.col('Departure'), "dd.MM.yyyy HH:mm").cast('timestamp'))<=20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
# Rank stops by departure_time for every trip and day 
from pyspark.sql import Window
trip_window = Window.partitionBy('Trip_ID','Day').orderBy(F.asc('Departure'))
trip_rank = F.rank().over(trip_window).alias('stop')
begin = df_max_hour.select('*', trip_rank).alias('begin').orderBy('Trip_ID','Arrival','Departure').fillna(0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
begin.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+-------------------+----------+-------+----+----------+----------------+----------------+-----+-------+-----------+----------+----+
|        Trip_ID|        Station_Name|       Arrival_Real|       Day|Line_ID|Type|Station_ID|       Departure|         Arrival|Total|Delayed|Probability|Mean_Delay|stop|
+---------------+--------------------+-------------------+----------+-------+----+----------+----------------+----------------+-----+-------+-----------+----------+----+
|85:11:17215:001|             Muttenz|13.05.2019 05:04:55|13.05.2019|  17215| Zug|   8500020|13.05.2019 05:03|13.05.2019 05:03|    4|      4|        1.0|     79.25|   1|
|85:11:17215:001|            Pratteln|13.05.2019 05:08:38|13.05.2019|  17215| Zug|   8500021|13.05.2019 05:07|13.05.2019 05:07|    4|      4|        1.0|     88.75|   2|
|85:11:17215:001|Pratteln Salina R...|13.05.2019 05:10:39|13.05.2019|  17215| Zug|   8517131|13.05.2019 05:09|13.05.2019 05:09|    4|      4|        1

In [24]:
# Create dataframe for every connection 
end = begin.drop('Departure').withColumn('stop', begin.stop -1).alias('end')
data = begin.drop('Arrival','Mean_Delay','Probability').join(end, on=['stop','Day','Trip_ID','Type','Line_ID'])\
            .orderBy('Trip_ID','Arrival','Departure').drop('stop')\
            .toDF('Day','Trip_ID','Type','Line_ID','Start_Station','Start_ID','Start_Time',
                  'Stop_Station','Stop_ID','Stop_Time','Propability','Mean_Delay')\
            

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Day: string (nullable = true)
 |-- Trip_ID: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Line_ID: string (nullable = true)
 |-- Start_Station: string (nullable = true)
 |-- Start_ID: string (nullable = true)
 |-- Start_Time: string (nullable = true)
 |-- Stop_Station: string (nullable = true)
 |-- Stop_ID: string (nullable = true)
 |-- Stop_Time: string (nullable = true)
 |-- Propability: double (nullable = false)
 |-- Mean_Delay: double (nullable = false)

### Save final dataframe in CSV file

In [ ]:
%%spark -o df_stations -n -1

In [18]:
%%local
data.to_csv("../data/Zurich_TransportConnections_WORKINGDAY.csv", index=False)

### Use stop-times table (for what?)

In [ ]:
#trips_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/trips")
#calendar_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/calendar")
#routes_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/routes")
stop_times_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stop_times")
stop_times_df.show(5)

In [ ]:
relevant_times = stop_times_df.join(ids,stop_times_df['stop_id']==ids['_c0'])\
                .select("stop_sequence","stop_id","arrival_time","departure_time")\
                .orderBy('stop_id','arrival_time','departure_time','stop_sequence').dropDuplicates()
relevant_times.show(5)